In [ ]:
import numpy as np
from pandas import read_csv
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.svm import OneClassSVM

In [ ]:
X_train = read_csv("../data/PAMAP2/x_train_data.csv")
X_test = read_csv("../data/PAMAP2/x_test_data.csv")
y_train = read_csv("../data/PAMAP2/y_train_data.csv")
y_test = read_csv("../data/PAMAP2/y_test_data.csv")

X_train["activity"] = y_train  # First 80% of the data
X_test["activity"] = y_test  # Last 20% of the data

MIN_SAMPLES = X_train["activity"].value_counts().sort_values().iloc[3]

In [ ]:
def score_function(model: OneClassSVM, X_train, y_true) -> float:
    return float(f1_score(np.ones(len(y_true)), model.predict(X_train), pos_label=1))

In [ ]:
grid_search: GridSearchCV[OneClassSVM] = GridSearchCV(
    estimator=OneClassSVM(kernel="rbf"),
    param_grid={
        "nu": [0.01, 0.05, 0.1, 0.2, 0.3, 0.5],
        "gamma": [0.001, 0.01, 0.1, 0.5, 1],
    },
    scoring=make_scorer(score_func=score_function),
    n_jobs=-1,
    verbose=3,
    cv=5,
).fit(X_train)

In [ ]:
random_search = RandomizedSearchCV(
    estimator=OneClassSVM(kernel="rbf"),
    param_distributions={
        "nu": [0.01, 0.025, 0.05, 0.1, 0.2, 0.3, 0.5],
        "gamma": [0.001, 0.01, 0.025, 0.1, 0.25, 0.5, 1],
    },
    n_iter=30,
    scoring=make_scorer(score_func=score_function),
    n_jobs=-1,
    cv=5,
    verbose=3,
    random_state=42,
).fit(X_train)

In [ ]:
print("Grid Search Best Params:", grid_search.best_params_)
print("Random Search Best Params:", random_search.best_params_)